# Laboratorio 2 – Simulación de Montecarlo

Este laboratorio aplica **Simulación de Montecarlo** para resolver dos problemas distintos:

1. **Planeación de ahorro universitario**  
   Modelar el crecimiento de un fondo de ahorro a lo largo de 17 años con aportes anuales fijos y rendimientos variables siguiendo una distribución normal. Se busca estimar resultados promedio, escenarios extremos y comportamiento del ahorro en el tiempo.

2. **Optimización de presupuesto publicitario**  
   Usar datos históricos de inversión en TV, Radio y Periódico para construir un modelo de regresión que relacione la inversión con las ventas. Posteriormente, simular distintas combinaciones de inversión para identificar la estrategia que maximice las ventas.

El desarrollo se realizará en **Python** mediante un **Jupyter Notebook** y se complementará con una **aplicación en Streamlit** para visualizar e interactuar con los resultados.

## Ejercicio 1 – Ahorro Universitario con Rendimiento Variable

**Objetivo:**  
Modelar y simular el crecimiento de un fondo de ahorro durante 17 años para determinar la probabilidad de alcanzar una meta financiera de **$500,000**.

**Escenario planteado:**  
- Juan y Juana Doe aportarán **$20,000** al inicio de cada año.  
- La tasa de rendimiento anual sigue una **distribución normal** con:
  - Media: 4% (0.04)
  - Desviación estándar: 10% (0.10)
- Cada simulación debe generar 17 rendimientos anuales independientes.
- Se realizarán al menos **1,000 simulaciones**.

**Resultados esperados:**
1. Rendimiento promedio total al final de los 17 años.
2. Monto promedio acumulado al final de los 17 años.
3. Escenario pesimista y escenario optimista.
4. Gráficas:
   - Rendimientos obtenidos por año.
   - Monto ahorrado por año.
   - Monto acumulado por año.

**Método sugerido:**
- Utilizar **Simulación de Montecarlo** para modelar el crecimiento año a año.
- Generar tasas aleatorias de rendimiento en cada simulación.
- Acumular el monto considerando aportes + rendimiento compuesto.